# Step 3: Scrape MyAnimeList Information

**Metis Project 2, Andrew Zhou**

Now that we've linked the anime from our sales database to MyAnimeList, we scrape all the information we want from MAL.

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import pickle
import time
import sys

sys.path.append('..')

from utilities.scraping_utilities import \
    get_anime_link, get_anime_data, create_mal_info_df

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [3]:
anime_sales_df = pd.read_pickle("../data/anime_sales_df_matched.pickle")
studio_df = pd.read_pickle("../data/studio_df.pickle")

In [4]:
studio_df

,link,anime_info
8bit,https://myanimelist.net/anime/producer/441/8bit,{'Tensei shitara Slime Datta Ken': 'https://my...
A-1 Pictures,https://myanimelist.net/anime/producer/56/A-1_...,{'Sword Art Online': 'https://myanimelist.net/...
A-Real,https://myanimelist.net/anime/producer/1257/A-...,{'Kenka Banchou Otome: Girl Beats Boys': 'http...
A.C.G.T.,https://myanimelist.net/anime/producer/179/ACGT,{'Freezing': 'https://myanimelist.net/anime/93...
Acca effe,https://myanimelist.net/anime/producer/2085/Ac...,{'Strike Witches: 501 Butai Hasshin Shimasu!':...
...,...,...
Wolfsbane,https://myanimelist.net/anime/producer/2052/Wo...,{'Peter Grill to Kenja no Jikan': 'https://mya...
Xebec,https://myanimelist.net/anime/producer/27/Xebec,{'To LOVE-Ru': 'https://myanimelist.net/anime/...
Yumeta Company,https://myanimelist.net/anime/producer/96/Yume...,{'Akkun to Kanojo': 'https://myanimelist.net/a...
Zero-G,https://myanimelist.net/anime/producer/1379/Ze...,{'Grand Blue': 'https://myanimelist.net/anime/...


Match anime in the sales data to their MAL links. Use fuzzy matching because
some names are slightly different. Some anime fail to be matched, and we drop them.

In [34]:
def get_anime_link_helper(x):
    """
    Helper for DataFrame.apply's function parameter below. Preferred to
    lambda function because of clarity.
    """
    studio_anime_info = studio_df.loc[x["studio"]]["anime_info"]
    return get_anime_link(x["title"], studio_anime_info, 
                          fuzzy_match = True, ratio = 90)

anime_sales_df["link"] = anime_sales_df.apply(get_anime_link_helper, axis=1)
anime_sales_df = anime_sales_df.dropna(subset=["link"])

In [30]:
keep_cols = ["episodes", "broadcast", "genres", "duration", "rating",\
        "score", "members", "favorites"]

mal_info_df = create_mal_info_df(anime_sales_df[["title", "link"]], keep_cols)

mal
#get_anime_data("https://myanimelist.net/anime/37779/Yakusoku_no_Neverland", keep_cols)

Getting: .hack//SIGN
Getting: .hack//Tasogare no Udewa Densetsu
Exception in getting anime information. Returning incomplete dataframe early. Potentially a connection issue.


In [35]:
anime_sales_df.to_pickle("../data/anime_sales_df_linked.pickle") 
mal_info_df.to_pickle("../data/mal_info_df.pickle")

In [175]:
#with open('../data/clean_anime_dicts.pickle', 'wb') as to_write:
#    pickle.dump(anime_dicts, to_write)